# 기타 웹 크롤링 프로젝트

1. 네이버 연관 검색어 수집
2. Gmarket 상품 데이터 수집

---

#### 최종 DataFrame 전처리 부분 팁
1. apply 메서드 활용
2. re 정규식 활용

## 0.라이브러리 불러오기

In [66]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from datetime import datetime
import re

## 1.네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [9]:
def naver_reate_keyowrds(query):
    url = f'https://search.naver.com/search.naver?query={query}' 
    response = requests.get(url) 
    dom = BeautifulSoup(response.text, 'html.parser') 
    elements = dom.select('.lst_related_srch > .item')
    keywords = [element.select_one('.tit').text for element in elements]
    df = pd.DataFrame({'keyowrd':keywords})
    df['query'] = query
    
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    df['datetime'] = now
    return df

<img src="../img/6.naver_keywords_1.png" height="80%" width="80%">

In [11]:
naver_reate_keyowrds('삼성전자')

,keyowrd,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:28
1,삼성전자 배당금,삼성전자,2023-08-25 12:28
2,삼성전자서비스,삼성전자,2023-08-25 12:28
3,삼성전자주식,삼성전자,2023-08-25 12:28
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:28
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:28
6,삼성전자몰,삼성전자,2023-08-25 12:28
7,삼성 전자레인지,삼성전자,2023-08-25 12:28
8,삼성전자 채용,삼성전자,2023-08-25 12:28
9,삼성전자 고객센터,삼성전자,2023-08-25 12:28


In [16]:
naver_reate_keyowrds('의자')

,keyowrd,query,datetime
0,책상 의자,의자,2023-08-25 12:30
1,공부 의자,의자,2023-08-25 12:30
2,명품 의자,의자,2023-08-25 12:30
3,푹신한 의자,의자,2023-08-25 12:30
4,팔걸이 없는 의자,의자,2023-08-25 12:30
5,허리 편한 의자,의자,2023-08-25 12:30
6,도서관 의자,의자,2023-08-25 12:30
7,튼튼한 의자,의자,2023-08-25 12:30
8,높낮이 의자,의자,2023-08-25 12:30
9,대기용 의자,의자,2023-08-25 12:30


---

## 2. Gmarket 상품 데이터 수집
- 베스트 상품 200개 데이터 수집
- 이미지 데이터도 수집하기

In [95]:
# 데이터 전처리
def processing_price(data):
    if len(re.findall('[0-9,]+', data)) >0:
        return re.findall('[0-9,]+', data)[0].replace(",",'')
    else:
        return False
    
# 데이터 수집
def Gmarket_best_data():
    url = f'https://www.gmarket.co.kr/n/best'
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select('#gBestWrap > div.best-list > ul > li')
    
    items = []
    for element in elements:
        items.append({
            'title' : element.select_one('.itemname').text,
            'link' : element.select_one('.itemname').get('href'),
            'img' : 'http:'+ element.select_one('.image__lazy').get('src'),
            'o_price' : element.select_one('.o-price').text,
            's_price' : element.select_one('.s-price').text.split(' ')[0],
        })
    
    df = pd.DataFrame(items)
    
    # 마지막 전처리
    df1 = df.copy() # 깊은 복사
    df1['o_price'] = df1['o_price'].apply(processing_price) # 숫자 데이터만 가져오기
    df1['s_price'] = df1['s_price'].apply(processing_price) # 숫자 데이터만 가져오기
    df1['o_price'][df1['o_price'] == False] = df1['s_price'][df1['o_price'] == False] # 데이터 결측치 채워주기
    
    df1['o_price'] = df1['o_price'].astype('int') # 숫자타입으로 변환
    df1['s_price'] = df1['s_price'].astype('int') # 숫자타입으로 변환

    # 할인율 컬럼 추가
    df1['discount rate'] = np.round((1 - df1['s_price'] / df1['o_price']) * 100, 2) # 할인율 컬럼 추가
    
    return df1

<img src="../img/6.naver_keywords_2.png" height="50%" width="50%">

In [96]:
df = Gmarket_best_data()
df

,title,link,img,o_price,s_price,discount rate
0,네추럴라이즈 키즈 멀티비타민 C/D 60꾸미 3종 3병,http://item.gmarket.co.kr/Item?goodscode=31130...,http://gdimg.gmarket.co.kr/3113057185/still/30...,22900,21760,4.98
1,[CGV](CGV) G마켓 예매권(금요특가_8/25),http://item.gmarket.co.kr/Item?goodscode=30753...,http://gdimg.gmarket.co.kr/3075338164/still/30...,15000,9980,33.47
2,[팸퍼스]팸퍼스 아르모니 팬티형 5단계(20X4),http://item.gmarket.co.kr/Item?goodscode=22199...,http://gdimg.gmarket.co.kr/2219928058/still/30...,45900,45900,0.00
3,나이키 뉴발란스 스우시 6팩 크루삭스 운동 스포츠양말,http://item.gmarket.co.kr/Item?goodscode=21052...,http://gdimg.gmarket.co.kr/2105251652/still/30...,25800,12900,50.00
4,[노르딕내추럴](오플닷컴) 2개 프로 오메가3 1280 mg 650 EPA 450 ...,http://item.gmarket.co.kr/Item?goodscode=22416...,http://gdimg.gmarket.co.kr/2241600510/still/30...,103700,92460,10.84
...,...,...,...,...,...,...
195,[벡셀]건전지 AA 40개입 최근생산품,http://item.gmarket.co.kr/Item?goodscode=25039...,http://gdimg.gmarket.co.kr/2503938526/still/30...,21000,20900,0.48
196,[페리오]페리오 펌핑치약 285g 6개 스피아민트,http://item.gmarket.co.kr/Item?goodscode=16544...,http://gdimg.gmarket.co.kr/1654416099/still/30...,24400,24400,0.00
197,[롯데칠성]아이시스 8.0 300ml 20펫 3묶음(60펫),http://item.gmarket.co.kr/Item?goodscode=17527...,http://gdimg.gmarket.co.kr/1752706460/still/30...,18000,18000,0.00
198,[좋은느낌]좋은느낌 패드 BEST 모음/+견본 추가 증정,http://item.gmarket.co.kr/Item?goodscode=24839...,http://gdimg.gmarket.co.kr/2483949419/still/30...,18900,18900,0.00
